## A simple multinomial logit model using the Swissmetro dataset

In [1]:
import sys
sys.path.append('..')

In [2]:
# import packages
import pandas as pd

from pycmtensor import train
from pycmtensor import Dataset
from pycmtensor.expressions import Beta
import pycmtensor.optimizers as optim
import pycmtensor.scheduler as sched
from pycmtensor.models import MNL

# read the data and load it into a dataset
df = pd.read_csv('http://transp-or.epfl.ch/data/swissmetro.dat', sep='\t')

# remove unknown choices
df = df.loc[df["CHOICE"] > 0]

# Load the DataFrame into a Dataset object
ds = Dataset(df, choice="CHOICE")
ds.scale_variable("TRAIN_TT", 100)
ds.scale_variable("SM_TT", 100)
ds.scale_variable("CAR_TT", 100)
ds.scale_variable("TRAIN_CO", 100)
ds.scale_variable("SM_CO", 100)
ds.scale_variable("CAR_CO", 100)
ds.split(frac=0.80)

# Define the alternative specific constants (ASCs) for each mode of transport
ASC_TRAIN = Beta("ASC_TRAIN", 0., None, None, 0)
ASC_SM = Beta("ASC_SM", 0., None, None, 0)
ASC_CAR = Beta("ASC_CAR", 0., None, None, 1)

# Define the coefficients for the variables
B_COST = Beta("B_COST", 0., None, None, 0)
B_TIME_TRAIN = Beta("B_TIME_TRAIN", 0., None, None, 0)
B_TIME_SM = Beta("B_TIME_SM", 0., None, None, 0)
B_TIME_CAR = Beta("B_TIME_CAR", 0., None, None, 0)
B_SEAT = Beta("B_SEAT", 0., None, None, 0)

# Define the utility functions for each mode of transport
V_TRAIN = (
	ASC_TRAIN 
	+ B_TIME_TRAIN * ds["TRAIN_TT"] 
	+ B_COST * ds["TRAIN_CO"]
)
V_SM = (
	ASC_SM 
	+ B_TIME_SM * ds["SM_TT"] 
	+ B_COST * ds["SM_CO"] 
	+ B_SEAT * ds["SM_SEATS"]
)
V_CAR = (
	ASC_CAR 
	+ B_TIME_CAR * ds["CAR_TT"] 
	+ B_COST * ds["CAR_CO"]
)

# Define the model
U = [V_TRAIN, V_SM, V_CAR]
AV = [ds["TRAIN_AV"], ds["SM_AV"], ds["CAR_AV"]]
model = MNL(ds, locals(), U, AV)

13:33:00 [INFO] CHOICE: {0: 1423, 1: 6216, 2: 3080}
13:33:00 [INFO] seed: 100 n_train_samples:8575 n_valid_samples:2144
13:33:06 [INFO] inputs in MNL: [TRAIN_AV, CAR_AV, SM_AV, TRAIN_TT, TRAIN_CO, SM_TT, SM_CO, SM_SEATS, CAR_TT, CAR_CO]
13:33:06 [INFO] Build time = 00:00:06


Estimate the model

In [3]:
# Train model
model.reset_values()
train(model, ds, optimizer=optim.Adam, lr_scheduler=sched.ConstantLR, batch_size=0, max_epochs=2000, base_learning_rate=0.01, convergence_threshold=1e-3, acceptance_method=0)

13:33:43 [INFO] Start (n=8575, epoch=0, NLL=-8869.17, error=86.29%)
13:33:43 [INFO] Train 0/2K (epoch=0, LL=-8735.75, error=42.77%, gnorm=2.646e-02)
13:33:45 [INFO] Train 126/2K (epoch=126, LL=-7269.83, error=42.72%, gnorm=7.075e-03)
13:33:46 [INFO] Train 209/2K (epoch=209, LL=-7156.10, error=40.16%, gnorm=5.178e-03)
13:33:47 [INFO] Train 291/2K (epoch=291, LL=-7103.39, error=38.11%, gnorm=3.709e-03)
13:33:48 [INFO] Train 376/2K (epoch=376, LL=-7076.64, error=37.73%, gnorm=2.621e-03)
13:33:49 [INFO] Train 464/2K (epoch=464, LL=-7062.16, error=37.22%, gnorm=1.939e-03)
13:33:51 [INFO] Train 600/2K (epoch=600, LL=-7050.68, error=37.17%, gnorm=1.407e-03)
13:33:53 [INFO] Train 743/2K (epoch=743, LL=-7044.91, error=37.17%, gnorm=1.055e-03)
13:33:53 [INFO] Train 768/2K (epoch=768, LL=-7044.28, error=37.08%, gnorm=9.986e-04)
13:33:53 [INFO] Model converged (t=10.107)
13:33:53 [INFO] Best results obtained at epoch 474: LL=-7060.98, error=37.08%, gnorm=1.88409e-03


Display model results:

In [ ]:
display(model.results.beta_statistics())
display(model.results.model_statistics())
display(model.results.benchmark())

Display model correlation matrices

In [ ]:
display(model.results.model_correlation_matrix())
# display(model.results.model_robust_correlation_matrix())

Plot model Loglikelihood, training and validation error

In [ ]:
model.results.show_training_plot()

Model validation prediction

In [ ]:
# predict CHOICE on the validation set and display the results.
# First three columns are the probabilities, the fourth column 
# is the predicted choice, fifth column is the actual choice.
pd.DataFrame(model.predict(ds))

Compute elasticities of SM_CO with respect to CHOICE:SM

In [ ]:
sm_co_wrt_sm = model.elasticities(ds, 1)["SM_CO"]
print(sm_co_wrt_sm.mean())

Plot the elasticities

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("ticks")

fig, ax = plt.subplots()
sns.histplot(sm_co_wrt_sm, kde=False, ax=ax, bins=40)

sns.despine()
plt.show()